### マルウェアのダウンロード
https://www.dropbox.com/s/ep8qjakfwh1rzk4/malimg_dataset.zip?dl=0  
を取得して同じディレクトリに格納し、解凍。  
マルウェアのファイルなので取り扱いは要注意、自己責任で。  
->マルウェアを画像化したpngとテキストファイルのみだった。  

In [ ]:
import os
import numpy as np
import imageio
import array
from IPython.display import Image
from PIL import Image as PIL_Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from pprint import pprint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

### 実行ファイルのグレースケール画像化

In [ ]:
filename = 'pe'
f = open(filename, 'rb')
ln = os.path.getsize(filename)
width = 256
rem = ln % width
a = array.array("B")
a.fromfile(f, ln - rem)
g = np.reshape(a, (len(a) // width, width))
g = np.uint8(g)
imageio.imwrite('putty.png', g)

In [ ]:
# 画像の確認
Image(filename='putty.png')

### マルウェア画像の確認

In [ ]:
dimen = 32

# マルウェアファミリーのリスト(フォルダ名のリスト)
dir_path = './malimg_dataset/malimg_paper_dataset_imgs/'
sub_dir_list = [name for name in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, name))]
pprint(sub_dir_list)

In [ ]:
images = list()
labels = list()

for i in range(len(sub_dir_list)):
    label = i
    image_names = os.listdir(dir_path + sub_dir_list[i])
    for image_path in image_names:
        path = dir_path + sub_dir_list[i] + '/' + image_path
        image = PIL_Image.open(path).convert('L')
        resize_image = image.resize((dimen, dimen))
        array = list()
        for x in range(dimen):
            sub_array = list()
            for y in range(dimen):
                sub_array.append(resize_image.load()[x, y])
            array.append(sub_array)
        image_data = np.array(array)
        image = np.array(np.reshape(image_data, (dimen, dimen, 1))) / 255
        images.append(image)
        labels.append(label)


In [ ]:
X = np.array(images)
y = np.array(to_categorical(np.array(labels), num_classes=len(sub_dir_list)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
input_shape = (dimen, dimen, 1)
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(sub_dir_list), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.1, batch_size=32, epochs=10)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('テストデータ精度: {:.3f} %'.format(score[1]*100))